In [ ]:
import pandas as pd

dataset = pd.read_csv("./sms-spam/collection.tsv", sep="\t", names=["classification", "text"])

In [ ]:
from phoenix.otel import register
import os

os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://localhost:6006"
# configure the Phoenix tracer
tracer_provider = register(
    auto_instrument=True,
    protocol="http/protobuf",
    project_name="dspy"
)

Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: dspy
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.



In [18]:
import dspy
lm = dspy.LM('openai/gpt-4o-mini', api_key=input("API KEY"))
dspy.configure(lm=lm)

In [33]:

from typing import Literal

class TextClassifier(dspy.Signature):
    text: str = dspy.InputField()
    classification: Literal['spam', 'ham'] = dspy.OutputField(desc="whether a text message is legitimate (ham) or spam")


classify = dspy.Predict(TextClassifier)

In [26]:

eval_set = [dspy.Example(text=row["text"], classification=row["classification"]).with_inputs("text") for i, row in dataset.sample(20).iterrows()]
train_set = [dspy.Example(text=row["text"], classification=row["classification"]).with_inputs("text") for i, row in dataset.sample(200).iterrows()]

In [60]:
my_metric = lambda expected, actual, _ = None: actual.classification == expected.classification
evaluate = dspy.Evaluate(
    devset=eval_set,
    metric=my_metric,
    max_errors=1
)


In [61]:
evaluate(classify, display_table=True, display_progress=True)

Average Metric: 17.00 / 20 (85.0%): 100%|██████████| 20/20 [00:10<00:00,  1.88it/s] 

2025/04/06 11:43:15 INFO dspy.evaluate.evaluate: Average Metric: 17 / 20 (85.0%)


,text,example_classification,pred_classification,<lambda>
0,Sorry i din lock my keypad.,ham,ham,✔️ [True]
1,Aiyah then i wait lor. Then u entertain me. Hee...,ham,ham,✔️ [True]
2,Ok lor... But buy wat?,ham,ham,✔️ [True]
3,Ooooooh I forgot to tell u I can get on yoville on my phone,ham,ham,✔️ [True]
4,Dad says hurry the hell up,ham,ham,✔️ [True]
5,First has she gained more than &lt;#&gt; kg since she took in. Sec...,ham,ham,✔️ [True]
6,December only! Had your mobile 11mths+? You are entitled to update...,spam,spam,✔️ [True]
7,You intrepid duo you! Have a great time and see you both soon.,ham,ham,✔️ [True]
8,Still i have not checked it da. . .,ham,ham,✔️ [True]
9,When u love someone Dont make them to love u as much as u do. But ...,ham,ham,✔️ [True]


85.0

In [37]:
few_shot = dspy.LabeledFewShot()

few_shot_compiled = few_shot.compile(student=classify, trainset=train_set)

In [62]:
evaluate(few_shot_compiled, display_progress=True, display_table=True)

Average Metric: 18.00 / 20 (90.0%): 100%|██████████| 20/20 [00:16<00:00,  1.23it/s] 

2025/04/06 11:43:43 INFO dspy.evaluate.evaluate: Average Metric: 18 / 20 (90.0%)


,text,example_classification,pred_classification,<lambda>
0,Sorry i din lock my keypad.,ham,ham,✔️ [True]
1,Aiyah then i wait lor. Then u entertain me. Hee...,ham,ham,✔️ [True]
2,Ok lor... But buy wat?,ham,ham,✔️ [True]
3,Ooooooh I forgot to tell u I can get on yoville on my phone,ham,ham,✔️ [True]
4,Dad says hurry the hell up,ham,ham,✔️ [True]
5,First has she gained more than &lt;#&gt; kg since she took in. Sec...,ham,ham,✔️ [True]
6,December only! Had your mobile 11mths+? You are entitled to update...,spam,spam,✔️ [True]
7,You intrepid duo you! Have a great time and see you both soon.,ham,ham,✔️ [True]
8,Still i have not checked it da. . .,ham,ham,✔️ [True]
9,When u love someone Dont make them to love u as much as u do. But ...,ham,ham,✔️ [True]


90.0

In [67]:
few_shot_compiled.save("./labeled_few_shot.json")

In [45]:
bootstrap_few_shot = dspy.BootstrapFewShot(max_labeled_demos=8, max_rounds=2)

bootstrap_few_shot_compiled = bootstrap_few_shot.compile(classify, trainset=train_set)

  2%|▏         | 4/200 [00:00<00:03, 55.13it/s]

Bootstrapped 4 full traces after 4 examples for up to 2 rounds, amounting to 4 attempts.


In [63]:
evaluate(bootstrap_few_shot_compiled, display_progress=True, display_table=True)

  0%|          | 0/20 [00:00<?, ?it/s]

Average Metric: 19.00 / 20 (95.0%): 100%|██████████| 20/20 [00:15<00:00,  1.31it/s] 

2025/04/06 11:44:22 INFO dspy.evaluate.evaluate: Average Metric: 19 / 20 (95.0%)


,text,example_classification,pred_classification,<lambda>
0,Sorry i din lock my keypad.,ham,ham,✔️ [True]
1,Aiyah then i wait lor. Then u entertain me. Hee...,ham,ham,✔️ [True]
2,Ok lor... But buy wat?,ham,ham,✔️ [True]
3,Ooooooh I forgot to tell u I can get on yoville on my phone,ham,ham,✔️ [True]
4,Dad says hurry the hell up,ham,ham,✔️ [True]
5,First has she gained more than &lt;#&gt; kg since she took in. Sec...,ham,ham,✔️ [True]
6,December only! Had your mobile 11mths+? You are entitled to update...,spam,spam,✔️ [True]
7,You intrepid duo you! Have a great time and see you both soon.,ham,ham,✔️ [True]
8,Still i have not checked it da. . .,ham,ham,✔️ [True]
9,When u love someone Dont make them to love u as much as u do. But ...,ham,ham,✔️ [True]


95.0

In [41]:
bootstrap_few_shot_compiled.save("./bootstrap-few-shot.json")

In [ ]:
mipro = dspy.MIPROv2(my_metric)

mipro_compiled = mipro.compile(classify, trainset=train_set)

2025/04/06 11:27:21 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/04/06 11:27:21 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/04/06 11:27:21 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=10 sets of demonstrations...


Bootstrapping set 1/10
Bootstrapping set 2/10
Bootstrapping set 3/10


 10%|█         | 4/40 [00:00<00:00, 40.49it/s]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/10


  5%|▌         | 2/40 [00:01<00:30,  1.26it/s]


...

2025/04/06 11:38:10 INFO dspy.evaluate.evaluate: Average Metric: 24 / 25 (96.0%)
2025/04/06 11:38:10 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 96.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 7', 'Predictor 0: Few-Shot Set 5'].
2025/04/06 11:38:10 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [96.0, 92.0, 96.0, 100.0, 96.0, 96.0, 96.0, 92.0, 96.0, 96.0, 88.0, 96.0, 96.0, 92.0, 92.0, 92.0, 92.0, 100.0, 92.0, 92.0, 96.0, 100.0, 96.0, 100.0, 96.0, 100.0, 92.0, 96.0, 96.0]
2025/04/06 11:38:10 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [94.38, 96.88, 95.0, 94.38]
2025/04/06 11:38:10 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 96.88
2025/04/06 11:38:10 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/04/06 11:38:10 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 34 / 34 - Full Evaluation =====
2025/04/06 11:38:10 INFO dspy.teleprompt.mipro_optimizer_v2: 


Average Metric: 58.00 / 60 (96.7%):  38%|███▊      | 60/160 [00:34<01:21,  1.22it/s] 

2025/04/06 11:38:46 ERROR dspy.utils.parallelizer: Error for Example({'text': 'YOUR CHANCE TO BE ON A REALITY FANTASY SHOW call now = 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national = rate call.', 'classification': 'spam'}) (input_keys={'text'}): litellm.RateLimitError: RateLimitError: OpenAIException - Rate limit reached for gpt-4o-mini in organization org-Cj9hZ8dens3JYF8ae7IcNsL0 on tokens per min (TPM): Limit 200000, Used 199303, Requested 1409. Please try again in 213ms. Visit https://platform.openai.com/account/rate-limits to learn more.. Set `provide_traceback=True` for traceback.


Average Metric: 155.00 / 159 (97.5%): 100%|██████████| 160/160 [01:26<00:00,  1.84it/s]

2025/04/06 11:39:37 INFO dspy.evaluate.evaluate: Average Metric: 155.0 / 160 (96.9%)
2025/04/06 11:39:37 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [94.38, 96.88, 95.0, 94.38, 96.88]
2025/04/06 11:39:37 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 96.88
2025/04/06 11:39:37 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/04/06 11:39:37 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/04/06 11:39:37 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 96.88!


In [64]:
evaluate(mipro_compiled, display_progress=True, display_table=True)

Average Metric: 20.00 / 20 (100.0%): 100%|██████████| 20/20 [00:12<00:00,  1.54it/s]

2025/04/06 11:45:43 INFO dspy.evaluate.evaluate: Average Metric: 20 / 20 (100.0%)


,text,example_classification,pred_classification,<lambda>
0,Sorry i din lock my keypad.,ham,ham,✔️ [True]
1,Aiyah then i wait lor. Then u entertain me. Hee...,ham,ham,✔️ [True]
2,Ok lor... But buy wat?,ham,ham,✔️ [True]
3,Ooooooh I forgot to tell u I can get on yoville on my phone,ham,ham,✔️ [True]
4,Dad says hurry the hell up,ham,ham,✔️ [True]
5,First has she gained more than &lt;#&gt; kg since she took in. Sec...,ham,ham,✔️ [True]
6,December only! Had your mobile 11mths+? You are entitled to update...,spam,spam,✔️ [True]
7,You intrepid duo you! Have a great time and see you both soon.,ham,ham,✔️ [True]
8,Still i have not checked it da. . .,ham,ham,✔️ [True]
9,When u love someone Dont make them to love u as much as u do. But ...,ham,ham,✔️ [True]


100.0

In [57]:
mipro_compiled.save("./miprov2.json")